<a href="https://colab.research.google.com/github/AbooMardiiyah/Hugging-face-transformers/blob/main/Hausa_speech_recog_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<hr>

# Prepare environnement and load datasets

<hr>

## Install dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install jiwer
!pip install datasets==1.13.3
!pip install transformers==4.12.5
!pip install huggingface_hub==0.1
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install librosa==0.8.1

In [3]:
import re
import os
import json

import torch
import librosa
import numpy as np
from transformers import (
    Trainer,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    TrainingArguments,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor
    
)
from datasets import (
    Dataset,
    load_from_disk,
    load_dataset,
    load_metric,
    concatenate_datasets
)

from dataclasses import dataclass, field
from typing import (
    Any,
    Dict,
    List,
    Optional,
    Union
)
from joblib import Parallel, delayed

import warnings
warnings.filterwarnings("ignore")
#os.environ["TOKENIZERS_PARALLELISM"] = "true"


## Set up data, model and traning parameters

In [4]:
model_checkpoint =  "facebook/wav2vec2-xls-r-300m"

MODEL_NAME = "4-ha-wa2vec-data-aug-xls"+ model_checkpoint.split('xls')[-1]
MODEL_NAME

'4-ha-wa2vec-data-aug-xls-r-300m'

In [7]:
!tar -xzvf  "/content/drive/MyDrive/cv-corpus-7.0-2021-07-21-ha.tar.gz" -C "./"

cv-corpus-7.0-2021-07-21/ha/
cv-corpus-7.0-2021-07-21/ha/clips/
cv-corpus-7.0-2021-07-21/ha/dev.tsv
cv-corpus-7.0-2021-07-21/ha/invalidated.tsv
cv-corpus-7.0-2021-07-21/ha/other.tsv
cv-corpus-7.0-2021-07-21/ha/test.tsv
cv-corpus-7.0-2021-07-21/ha/train.tsv
cv-corpus-7.0-2021-07-21/ha/validated.tsv
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26692998.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693000.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693002.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693005.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693006.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693872.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693873.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693874.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693875.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26693876.mp3
cv-corpus-7.0-2021-07-21/ha/clips/common_voice_ha_26694102.mp3
cv-corpu

In [10]:
#check how many audoio files are in there
len(os.listdir("./cv-corpus-7.0-2021-07-21/ha/clips"))

2511

In [11]:
# Data parameters
# Change "BASE_DIRE" according to your data folder 
# which contain audio clips and utterance files 
# (train.tsv, other.tsv and test.tsv) from commovoice dataset
BASE_DIR = "./cv-corpus-7.0-2021-07-21/ha"
CLIPS_DIR = "clips"
TRAIN_FILE = "train.tsv"
VALID_FILE = "other.tsv"
# We load test dataset from commonvoice file 
# only for delete its utterances contain in valid dataset (othe.tsv)
TEST_FILE = "test.tsv"


# Model paramaters
# Pretrain model chektpoint from HuggingFace
model_checkpoint =  "facebook/wav2vec2-xls-r-300m"
attention_dropout = 0.0
hidden_dropout = 0.1 
layerdrop = 0.1
ctc_loss_reduction = "mean"

MODEL_NAME = "4-ha-wa2vec-data-aug-xls"+ model_checkpoint.split('xls')[-1]

# Training parameters
num_train_epochs = 30
steps = 30
save_steps = 5 * steps
lr = 1e-4
batch_size = 16
gradient_accumulation_steps = 2

## Load data

In [12]:
train = Dataset.from_csv(os.path.join(BASE_DIR, TRAIN_FILE), sep="\t")
valid = Dataset.from_csv(os.path.join(BASE_DIR, VALID_FILE), sep="\t")
test = Dataset.from_csv(os.path.join(BASE_DIR, TEST_FILE), sep="\t")

Using custom data configuration default-7a50f6d0e87901d5


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7a50f6d0e87901d5/0.0.0. Subsequent calls will reuse this data.


Using custom data configuration default-c734b91678e3ecdf


...

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c734b91678e3ecdf/0.0.0. Subsequent calls will reuse this data.


Using custom data configuration default-8372100d9cfae404


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8372100d9cfae404/0.0.0. Subsequent calls will reuse this data.


In [13]:
# We remove all columns except "path" which contain audio path 
# and "sentence" which contain its transcription
train = train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
valid = valid.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
test = test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [14]:
print("Data feature", valid.column_names)
print("Train shape", train.shape)
print("Valid shape", valid.shape)
print("Test shape", test.shape)

Data feature ['path', 'sentence']
Train shape (1386, 2)
Valid shape (938, 2)
Test shape (149, 2)


In [15]:
def tolower(batch):
    batch["sentence"] = batch["sentence"].lower()
    return batch

train = train.map(tolower)
valid = valid.map(tolower)
test = test.map(tolower)

  0%|          | 0/1386 [00:00<?, ?ex/s]

  0%|          | 0/938 [00:00<?, ?ex/s]

  0%|          | 0/149 [00:00<?, ?ex/s]

In [16]:
# filter the validation data to ensure that the same utterances do not appear in the test data
valid_size = len(valid)
valid = valid.filter(lambda x: x not in test["sentence"],
                     input_columns=["sentence"])
valid = valid.filter(lambda x: x not in test["path"],
                     input_columns=["path"])
try:
    assert len(valid) == valid_size
except:
    print("test data utterances will be removed in valid data")

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

<hr>

# Data preprocessing

<hr>

## Remove special characters

In [17]:
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\—]'

def remove_special_characters(batch):
    tmp = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    batch["sentence"] = re.sub("[ʻ’]", "'", tmp)
    return batch

train = train.map(remove_special_characters)
valid = valid.map(remove_special_characters)

  0%|          | 0/1386 [00:00<?, ?ex/s]

  0%|          | 0/938 [00:00<?, ?ex/s]

## Create vocabulary

Which is all unique charactere on the corpus.

In [18]:
# Creating the vocabulary for the model
vocab_dict = set(" ".join(valid['sentence'] + train['sentence']).lower())
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_dict))}
vocab_dict

{' ': 0,
 "'": 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27,
 'í': 28,
 'ƙ': 29,
 'ƴ': 30,
 'ɓ': 31,
 'ɗ': 32}

In [19]:
# We add special characters for padding [PAD] and 
# unknow [UNK] char and use '|' as space char
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

In [20]:
len(vocab_dict)

35

In [21]:
# Save vocab file as a json file
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

## Change train and valid data distribution

In [22]:
# Select 700 utterances from valid dataset
# and add it on train dataset.
valid = valid.shuffle(seed=120)
tmp = valid.select(range(700))

# Concat for loop
from tqdm import tqdm
for i in tqdm(range(len(tmp))):
    train = train.add_item(tmp[i])


valid = valid.select(range(700, len(valid)))

100%|██████████| 700/700 [00:02<00:00, 298.66it/s]


In [23]:
# New data distribution
print("Train shape", train.shape)
print("Valid shape", valid.shape)
print("Test shape", test.shape)

Train shape (2086, 2)
Valid shape (238, 2)
Test shape (149, 2)


## Audio processing

In [24]:
def load_audio(path):
    """This function load audio and resample it if necessary"""
    speech_array, sampling_rate = librosa.load(path, sr=16000)
    # be sure, one never know
    if sampling_rate!=16000:
        speech_array = librosa.resample(
            y=np.asarray(speech_array), 
            orig_sr=sampling_rate,
            target_sr=16_000
        )
    return np.array(speech_array)


def build_audio_path(paths):
    """Path builder."""
    return [os.path.join(BASE_DIR, CLIPS_DIR, path) for path in paths]


def speech_file_to_array_fn(batch, n_jobs=-1):
    """
    Multi-processing for loading audio.
    Return list of numpy arrays.
    """
    audios = Parallel(n_jobs=n_jobs, verbose=0)(
        delayed(load_audio)(path) for path in build_audio_path(batch["path"])
        )
    batch["audio"] = audios
    return batch


In [25]:
train = train.map(speech_file_to_array_fn, batch_size=8, batched=True)
valid = valid.map(speech_file_to_array_fn, batch_size=8, batched=True)
train = train.shuffle(seed=120)

  0%|          | 0/261 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

## Create Tokenizer

In [27]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, 
                                             sampling_rate=16000, 
                                             padding_value=0.0, 
                                             do_normalize=True, 
                                             return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained(MODEL_NAME)

In [28]:
def prepare_dataset(batch):
    # batched output is "un-batched"
    batch["input_values"] = processor(batch["audio"], sampling_rate=16_000).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [29]:
train = train.map(prepare_dataset, remove_columns=train.column_names)
valid = valid.map(prepare_dataset, remove_columns=valid.column_names)

print("Data feature", valid.column_names)
print("Train shape", train.shape)
print("Valid shape", valid.shape)

  0%|          | 0/2086 [00:00<?, ?ex/s]

  0%|          | 0/238 [00:00<?, ?ex/s]

Data feature ['input_values', 'input_length', 'labels']
Train shape (2086, 3)
Valid shape (238, 3)


In [30]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

<hr>

# Training

<hr>

In [33]:
training_args = TrainingArguments(
    output_dir=MODEL_NAME,
    group_by_length=True,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    evaluation_strategy="steps",
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    fp16=True,
    save_steps=save_steps,
    eval_steps=steps,
    logging_steps=steps,
    learning_rate=lr,
    warmup_steps=2*steps,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False
)

In [34]:
model = Wav2Vec2ForCTC.from_pretrained(
    model_checkpoint, 
    attention_dropout=attention_dropout,
    hidden_dropout=hidden_dropout, 
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=layerdrop, 
    ctc_loss_reduction=ctc_loss_reduction,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_extractor()

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'project_hid.bias', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [35]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=valid,
    tokenizer=processor.feature_extractor
)

Using amp fp16 backend


In [36]:
# Run this for disable automatic wandb tracking in transformer
os.environ["WANDB_DISABLED"] = "true"

In [37]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 2086
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1950


Step,Training Loss,Validation Loss,Wer
30,14.579300,8.736645,1.000000
60,6.339300,3.765665,1.000000
90,3.588000,2.983178,1.000000
120,2.939200,2.843700,1.000000
150,2.886800,2.822742,1.000000
180,2.816500,2.828975,1.000000
210,2.842000,2.796848,1.000000
240,2.793700,2.784613,1.000000
270,2.832900,2.775942,1.000000
300,2.774500,2.772082,1.000000


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 238
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 238
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 238
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 238
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_le

TrainOutput(global_step=1950, training_loss=1.3241328425285144, metrics={'train_runtime': 8232.8371, 'train_samples_per_second': 7.601, 'train_steps_per_second': 0.237, 'total_flos': 8.368055357244446e+18, 'train_loss': 1.3241328425285144, 'epoch': 29.99})

In [38]:
trainer.save_model(MODEL_NAME)

Saving model checkpoint to 4-ha-wa2vec-data-aug-xls-r-300m
Configuration saved in 4-ha-wa2vec-data-aug-xls-r-300m/config.json
Model weights saved in 4-ha-wa2vec-data-aug-xls-r-300m/pytorch_model.bin
Configuration saved in 4-ha-wa2vec-data-aug-xls-r-300m/preprocessor_config.json


<hr>






















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































# Submission

<hr>

In [39]:
import pandas as pd
from jiwer import wer

In [40]:
!unzip "/content/drive/MyDrive/test_dataset.zip" -d "./"

Archive:  /content/drive/MyDrive/test_dataset.zip
  inflating: ./test_dataset/ID_0ETZSY8G.mp3  
  inflating: ./test_dataset/ID_0LR3J6PR.mp3  
  inflating: ./test_dataset/ID_0O4D8HOJ.mp3  
  inflating: ./test_dataset/ID_16AJ6WNU.mp3  
  inflating: ./test_dataset/ID_1CXWGKXN.mp3  
  inflating: ./test_dataset/ID_1D590RX7.mp3  
  inflating: ./test_dataset/ID_1FGVUFZJ.mp3  
  inflating: ./test_dataset/ID_1OXZ34UJ.mp3  
  inflating: ./test_dataset/ID_1SAQR925.mp3  
  inflating: ./test_dataset/ID_2AVRKO1I.mp3  
  inflating: ./test_dataset/ID_2WS33HCN.mp3  
  inflating: ./test_dataset/ID_2ZWA68QS.mp3  
  inflating: ./test_dataset/ID_344H7DH8.mp3  
  inflating: ./test_dataset/ID_35RGWK5L.mp3  
  inflating: ./test_dataset/ID_38XUOVKD.mp3  
  inflating: ./test_dataset/ID_3BWZBEQF.mp3  
  inflating: ./test_dataset/ID_3USAJCBS.mp3  
  inflating: ./test_dataset/ID_46TAMC5T.mp3  
  inflating: ./test_dataset/ID_57O9LDA5.mp3  
  inflating: ./test_dataset/ID_5S8BNVEN.mp3  
  inflating: ./test_dataset/ID

In [45]:
BASE_SUB_DIR = "./test_dataset"
SUB_CLIPS_DIR = "wav"
SUB_FILE = "SampleSubmission.csv"

def speech_file_to_array_sub(batch):
    audios = Parallel(n_jobs=-1, verbose=0)(
        delayed(load_audio)(path) for path in build_audio_path_sub(batch["Clip_ID"])
        )
    batch["audio"] = audios
    return batch

def build_audio_path_sub(paths):
    return [os.path.join(BASE_SUB_DIR, path+".mp3") for path in paths]

In [46]:
test = Dataset.from_csv(os.path.join(BASE_SUB_DIR, SUB_FILE))
test = test.map(speech_file_to_array_sub, batch_size=8, batched=True)

Using custom data configuration default-7bb12c95cc2da3db
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-7bb12c95cc2da3db/0.0.0)


  0%|          | 0/19 [00:00<?, ?ba/s]

In [47]:
test

Dataset({
    features: ['Clip_ID', 'Target', 'audio'],
    num_rows: 149
})

In [48]:
device = "cuda" #  "cpu" #
model = model.to(device)

In [60]:
def evaluate(batch):
    inputs = processor(batch["audio"], sampling_rate=16_000, return_tensors="pt", padding=True)
    with torch.no_grad():
            logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)
    batch["Target"] = processor.batch_decode(pred_ids, skip_special_tokens=True)
    return batch

In [61]:
result = test.map(evaluate, batched=True, batch_size=16)
result.remove_columns_("audio")
result

  0%|          | 0/10 [00:00<?, ?ba/s]

Dataset({
    features: ['Clip_ID', 'Target', 'pred_str'],
    num_rows: 149
})

Computing WER for test data


In [62]:
result['pred_str']

['jalal ya rubu ta labari mai tashin hankali da ke da ƙarshe mai farin ciki',
 'fasan banbanci sakanin silba da gangoni',
 'yana nin zafi a ƙirgi',
 'na yi tunanin zan same kowacan',
 'muna huta watsirara a cikin ya shi mai dume',
 'ina son jifi kasuwancin kasashin waje anan gaba',
 'ka kula da wannan labaran',
 'yisuf ya faɗawa lare yadda suka haɗu da hassan',
 'babangida ya ƙware a wasan tenis',
 'katan gargidan yarin za tu i bada kaliya ba',
 'ko akwai ciwa sosaa ƙirjin ku na ɓangara hagu',
 'na so wurar ran zama tsawon yaide shida mesa da cotin',
 'muna da abin mamaki',
 'lole tai rawa tare da gres',
 'an kama matashin ne da hannu cikin wani da kici',
 'sun ba wani abuda za su yi ranarijin mua',
 'habibu asabe hassan da aishanduk suna magana da faransanci',
 'ishaku bai damo da lare ba',
 'ya batun askrim me cakuleti bayan cin abinci',
 'an gutanir difhatin ban kwana jiyar sabida mr jons',
 'bun guda hakan jiyar',
 'za mu iya ba tare da taimakon ibrahim ba',
 'na san hauyan zu ya n

In [64]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=result["pred_str"], references=result["Target"])))

Test WER: 0.051


In [65]:
result.to_csv(MODEL_NAME + "-submition.csv", index=False)

13806